Mapping particle tracks from Ocean Parcels unto the Salish Sea Atlantis Boxes. 
Original code written by Bec Gordon & Javier Porobic, CSIRO.
Link to the [SSAM Ocean Parcels Repo](https://bitbucket.csiro.au/users/por07g/repos/ssam_oceanparcels/browse)

In [256]:
import os
import xarray as xr
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.path as mplPath
import shapefile
from netCDF4 import Dataset
import dateutil.parser
from shapely.geometry import Polygon, Point

In [296]:
shapefile_name = "/ocean/rlovindeer/Atlantis/ssam_oceanparcels/SalishSea/SalishSea_July172019_2/SalishSea_July172019.shp"
data_df = gpd.read_file(shapefile_name)
box_depth = data_df['BOTZ']
box_area = data_df['AREA']
box_volume = box_area * box_depth

#print(box_volume)

0                 0
1                 0
2                 0
3                 0
4       11621464000
           ...     
125    123069423000
126     94546607000
127               0
128               0
129     45304060750
Length: 130, dtype: int64


In [299]:
# Oil type properties & spill location selection

bitumen = {
    "Density": 1011.2, #kg/m^3
    "Naphthalene": 24, #mg/kg oil
    "Phenanthrene": 17,
    "Pyrene": 10,
    "Benzo": 3,
}

BunkerC = {
    "Density": 995.3,
    "Naphthalene": 680,
    "Phenanthrene": 796,
    "Pyrene": 266,
    "Benzo": 56,
}

Diesel = {
    "Density": 831.0,
    "Naphthalene": 3664,
    "Phenanthrene": 1000,
    "Pyrene": 0.000,
    "Benzo": 0.000,
}

Crude = {
    "Density": 884.7,
    "Naphthalene": 654,
    "Phenanthrene": 327,
    "Pyrene": 13,
    "Benzo": 2,
}

fuel_type = {
    "bitumen" : bitumen,
    "BunkerC" : BunkerC,
    "Diesel" : Diesel,
    "Crude" : Crude,
}

file_id = int(input( ))
scenario = {1 : "5b_Turn_Point_Diluted_bitumen",
            2 : "6a_VancouverHarbour_BunkerC",
            3 : "7a_JohnsonStrait_BunkerC",
            4 : "4a_ActivePass_Diesel",}

print("\nScenario running  :", scenario[file_id], sep = " ")


Scenario running  : 5b_Turn_Point_Diluted_bitumen


In [300]:
# Spill release times
release_start = '2020-07-01'  ## winter starts in December, Summer in Jul - Aug
release_end = '2020-07-02'
release_YYYYMM = '2020-01'
spill_volume = 2000 #m^3
oil_per_particle = (fuel_type[scenario[file_id].split(sep = '_')[-1]]["Density"] * spill_volume) / 100
release_start_time = np.datetime64(release_start)
#inputFileName = scenario[file_id] + str(release_start_time) + '_oil_disperse.nc'
inputFileName = '5b_Turn_Point_Diluted_bitumen2020-07-01_OP.nc'


In [301]:
numLayers = 7;
numSites = data_df.shape[0]
numTargetSites = numSites

#outputDT = 60*60
outputDT = 43100.00

stepsPerDay = int(86400.0/ outputDT);
numStepsPerDT = int(outputDT/3600.0)

debug = False

In [302]:
pfile = xr.open_dataset(str(inputFileName), decode_cf=True)

In [303]:
lon = np.ma.filled(pfile.variables['lon'], np.nan)
lat = np.ma.filled(pfile.variables['lat'], np.nan)
time = np.ma.filled(pfile.variables['time'], np.nan)
z = np.ma.filled(pfile.variables['z'], np.nan)
probs = np.ma.filled(pfile.variables['decay_value'], np.nan)

In [304]:
numParticles = lon.shape[0]

trackDates = [];

for i in range(0,numParticles):
    #print(time[i][0])
    #trackDates.append( dateutil.parser.parse(time[i][0]))
    trackDates.append(time[i][0])

RDiff = max(trackDates) - min(trackDates);
minDate = np.datetime64(release_start+"T00:30:00");
ts = pd.to_datetime(str(minDate));
d = ts.strftime('%Y-%m-%d %H:%M:%S');
print(d)

2020-07-01 00:30:00


In [305]:
numReleaseDays = 1;
numReleaseSteps = numReleaseDays * stepsPerDay;

trackLength = len(lon[0]);

print('trackLength = ' + str(trackLength))
print('numStepsPerDT = ' + str(numStepsPerDT))
numSteps = int(trackLength / numStepsPerDT)


trackLength = 145
numStepsPerDT = 11


In [312]:
# Create the netcdf output file

netcdfFileName = "Atlantis_" + scenario[file_id] + str(release_start_time) + "_volume2.nc"
try:
    os.remove(netcdfFileName)
except:
    pass
ncfile = Dataset(netcdfFileName, "w", format="NETCDF4", clobber=True)
Dataset.set_fill_on(ncfile)

# Dimensions
time = ncfile.createDimension("t", None)
b = ncfile.createDimension("b", numTargetSites)
z = ncfile.createDimension("z", numLayers)

In [313]:
# Variables
times = ncfile.createVariable("t",np.float64, ("t",))
oil = ncfile.createVariable("oil",np.float64,("t", "b"))
Naphthalene = ncfile.createVariable("Naphthalene",np.float64, ("t", "b", "z"),fill_value=0.0000,)
Phenanthrene = ncfile.createVariable("Phenanthrene",np.float64,("t", "b", "z"),fill_value=0.0000)
Pyrene = ncfile.createVariable("Pyrene",np.float64,("t", "b", "z"),fill_value=0.0000)
Benzo = ncfile.createVariable("Benzo",np.float64,("t", "b", "z"),fill_value=0.0000)

# Attributes
Naphthalene.units = "mgPAH/m^3"
Naphthalene.long_name = "Naphthalene"
Naphthalene.missing_value = 0.0000
Naphthalene.valid_min = 0.0000
Naphthalene.valid_max = 100000000.0

Phenanthrene.units = "mgPAH/m^3"
Phenanthrene.long_name = "Phenanthrene"
Phenanthrene.missing_value = 0.0000
Phenanthrene.valid_min = 0.0000
Phenanthrene.valid_max = 100000000.0

Pyrene.units = "mgPAH/m^3"
Pyrene.long_name = "Pyrene"
Pyrene.missing_value = 0.0000
Pyrene.valid_min = 0.0000
Pyrene.valid_max = 100000000.0

Benzo.units = "mgPAH/m^3"
Benzo.long_name = "Benzo(a)pyrene"
Benzo.missing_value = 0.0000
Benzo.valid_min = 0.0000
Benzo.valid_max = 100000000.0

oil.units = "kgOil/m^3"
oil.long_name = "Oil"

#times.units = "seconds since " + d
times.units = "seconds since 1950-01-01 00:00:00 +10"
times.dt = outputDT
times.long_name = "time"

# Populate variables with data
timeData = np.arange(0,(numSteps + numReleaseSteps)*outputDT,outputDT)
times[:] = timeData;

#Surface_particles = np.zeros((numSteps + numReleaseSteps, numTargetSites));
Dispersed_particles = np.zeros((numSteps + numReleaseSteps, numTargetSites, numLayers));


In [314]:
for partIndex in range(0, numParticles):

    trackDateDiff = trackDates[partIndex] - minDate;
    trackDateDiff = trackDateDiff/ np.timedelta64(1, 's')

    timeOffset = int(abs((trackDateDiff /outputDT)));

    for stepIndex in range(0, numSteps):
        timeValue = stepIndex + timeOffset

        partLon = lon[partIndex][stepIndex * numStepsPerDT];
        partLat = lat[partIndex][stepIndex * numStepsPerDT];
        partProb = probs[partIndex][stepIndex * numStepsPerDT];

        matchFound = 0;

        for targetIndex in range (0, numTargetSites):

            box_id = data_df.iloc[targetIndex].BOX_ID;
            box_coordinates = data_df.iloc[targetIndex].geometry;
            checks = box_coordinates.contains(Point(partLon, partLat));
            if box_volume[targetIndex] == 0:
                target_volume = 0
            else :
                target_volume = 1 / box_volume[targetIndex];
            
            if box_depth[targetIndex] < 26:
                layer = 0
            elif box_depth[targetIndex] == 50:
                layer = 1
            elif box_depth[targetIndex] == 100:
                layer = 2
            elif box_depth[targetIndex] == 200:
                layer = 3
            elif box_depth[targetIndex] > 200 and box_depth[targetIndex] < 401:
                layer = 4
            elif box_depth[targetIndex] > 400:
                layer = 5

            if checks:
                Dispersed_particles[timeValue][box_id][layer] = Dispersed_particles[timeValue][box_id][layer] + partProb * target_volume;

                # uncomment line below to ignore particle decay during debugging.
                # Dispersed_particles[timeValue][targetIndex] = Dispersed_particles[timeValue][targetIndex] + 1.0
            
                matchFound = 1
                if debug:
                    print('At time ' + str(timeValue) + ' Particle (' + str(partIndex) + ') in box ' + str(data_df.iloc[targetIndex].BOX_ID))

                break;

        if matchFound == 0:
            if debug:
                print('No match for particle')
                print(partLon, partLat)

        #break


#oil[:, :] = Dispersed_particles[:][:] * oil_per_particle;
Naphthalene[:,:,:] = Dispersed_particles * oil_per_particle * fuel_type[scenario[file_id].split(sep = '_')[-1]]["Naphthalene"];
Phenanthrene[:,:,:] = Dispersed_particles * oil_per_particle * fuel_type[scenario[file_id].split(sep = '_')[-1]]["Phenanthrene"];
Pyrene[:,:,:] = Dispersed_particles * oil_per_particle * fuel_type[scenario[file_id].split(sep = '_')[-1]]["Pyrene"];
Benzo[:,:,:] = Dispersed_particles * oil_per_particle * fuel_type[scenario[file_id].split(sep = '_')[-1]]["Benzo"];

ncfile.close()


In [233]:
np.histogram(Dispersed_particles)

(array([1799,    5,    4,    2,    1,    2,    0,    1,    1,    5]),
 array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.]))